In [1]:
%load_ext autoreload
%autoreload 2

In [32]:
import io
import json
import os
from pathlib import Path
from pprint import pprint
import requests
import sys
from typing import Optional

if '..' not in sys.path: sys.path.append('..')

import torch
from transformers import BertGenerationEncoder, BertGenerationDecoder, EncoderDecoderModel, BertTokenizer
from transformers.modeling_outputs import Seq2SeqLMOutput, BaseModelOutputWithPastAndCrossAttentions


# BERT Generator model
## Preload Encoder, Decoder and Tokenizer

In [7]:
model_name = 'google-bert/bert-base-uncased'

# leverage checkpoints for Bert2Bert model...
# use BERT's cls token as BOS token and sep token as EOS token
encoder: BertGenerationEncoder = BertGenerationEncoder.from_pretrained(model_name, bos_token_id=101, eos_token_id=102)
# add cross attention layers and use BERT's cls token as BOS token and sep token as EOS token
decoder: BertGenerationDecoder = BertGenerationDecoder.from_pretrained(
    model_name, add_cross_attention=True, is_decoder=True, bos_token_id=101, eos_token_id=102
)
tokenizer = BertTokenizer.from_pretrained(model_name)

You are using a model of type bert to instantiate a model of type bert-generation. This is not supported for all configurations of models and can yield errors.
You are using a model of type bert to instantiate a model of type bert-generation. This is not supported for all configurations of models and can yield errors.
Some weights of BertGenerationDecoder were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['bert.encoder.layer.0.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.0.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.0.crossattention.output.dense.bias', 'bert.encoder.layer.0.crossattention.output.dense.weight', 'bert.encoder.layer.0.crossattention.self.key.bias', 'bert.encoder.layer.0.crossattention.self.key.weight', 'bert.encoder.layer.0.crossattention.self.query.bias', 'bert.encoder.layer.0.crossattention.self.query.weight', 'bert.encoder.layer.0.crossattention.self.value.bias', 'bert.encoder.layer

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

## EncoderDecoderModel

In [ ]:
bert2bert = EncoderDecoderModel(encoder=encoder, decoder=decoder)

# create tokenizer...

input_ids = tokenizer(
    "This is a long article to summarize", add_special_tokens=False, return_tensors="pt"
).input_ids
labels = tokenizer("This is a short summary", return_tensors="pt").input_ids

# train...
loss = bert2bert(input_ids=input_ids, decoder_input_ids=labels, labels=labels).loss
loss.backward()

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

You are using a model of type bert to instantiate a model of type bert-generation. This is not supported for all configurations of models and can yield errors.


model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

You are using a model of type bert to instantiate a model of type bert-generation. This is not supported for all configurations of models and can yield errors.
Some weights of BertGenerationDecoder were not initialized from the model checkpoint at google-bert/bert-large-uncased and are newly initialized: ['bert.encoder.layer.0.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.0.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.0.crossattention.output.dense.bias', 'bert.encoder.layer.0.crossattention.output.dense.weight', 'bert.encoder.layer.0.crossattention.self.key.bias', 'bert.encoder.layer.0.crossattention.self.key.weight', 'bert.encoder.layer.0.crossattention.self.query.bias', 'bert.encoder.layer.0.crossattention.self.query.weight', 'bert.encoder.layer.0.crossattention.self.value.bias', 'bert.encoder.layer.0.crossattention.self.value.weight', 'bert.encoder.layer.1.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.1.crossattention.output.LayerNorm.weight

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/home/misha/miniconda3/envs/mllm/lib/python3.10/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:642: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


In [15]:
bert2bert.eval()
pred = bert2bert(input_ids=input_ids, decoder_input_ids=labels, labels=labels)
type(pred)

/home/misha/miniconda3/envs/mllm/lib/python3.10/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:642: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


transformers.modeling_outputs.Seq2SeqLMOutput

In [16]:
input_ids

tensor([[2023, 2003, 1037, 2146, 3720, 2000, 7680, 7849, 4697]])

In [17]:
gen_toks = bert2bert.generate(input_ids)
gen_toks

/home/misha/miniconda3/envs/mllm/lib/python3.10/site-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


tensor([[  101, 21297, 21297, 21297, 21297, 21297, 21297, 21297, 21297, 21976,
         21976, 21976,  5726,  5726,  5726,  5726,  5726,  5726,  5726,  5726]])

In [18]:
tokenizer.decode(gen_toks.squeeze().detach().cpu().numpy())

'[CLS]lomlomlomlomlomlomlomlom graded graded graded clay clay clay clay clay clay clay clay'


## Custom Bert Encoder -> Cls Embedding -> Decoder

In [17]:
tokenizer

BertTokenizer(name_or_path='google-bert/bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [18]:
s1 = 'How many words are needed to describe an ocean? Let\'s try to keep it short!'
toks1 = tokenizer(s1, return_tensors='pt')
toks1

{'input_ids': tensor([[ 101, 2129, 2116, 2616, 2024, 2734, 2000, 6235, 2019, 4153, 1029, 2292,
         1005, 1055, 3046, 2000, 2562, 2009, 2460,  999,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [19]:
s2 = 'Ocean deep and blue. It\'s endless and charming. Not much words, I\'d say.'
toks2 = tokenizer(s2, return_tensors='pt')
toks2

{'input_ids': tensor([[  101,  4153,  2784,  1998,  2630,  1012,  2009,  1005,  1055, 10866,
          1998, 11951,  1012,  2025,  2172,  2616,  1010,  1045,  1005,  1040,
          2360,  1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
enc_embs: BaseModelOutputWithPastAndCrossAttentions = encoder(toks1.input_ids)
enc_embs

BaseModelOutputWithPastAndCrossAttentions(last_hidden_state=tensor([[[-0.0525,  0.2416, -0.2183,  ..., -0.2659,  0.5905,  0.8504],
         [ 0.5397,  0.1460, -0.4024,  ...,  0.3013,  0.5229,  0.2699],
         [ 0.6455, -0.1244,  0.4163,  ..., -0.1054,  0.5575,  0.1057],
         ...,
         [ 0.7216, -0.1501, -0.1798,  ..., -0.1326,  0.2695, -0.0244],
         [ 0.1543,  0.5561,  0.2382,  ...,  0.4147,  0.2552,  0.2053],
         [ 0.8313,  0.1728, -0.4358,  ...,  0.1262, -0.4243, -0.3342]]],
       grad_fn=<NativeLayerNormBackward0>), past_key_values=None, hidden_states=None, attentions=None, cross_attentions=None)

In [ ]:
toks1.input_ids.shape, enc_embs.last_hidden_state.shape

(torch.Size([1, 21]), torch.Size([1, 21, 768]))

In [ ]:

encoder.config

BertGenerationConfig {
  "_name_or_path": "google-bert/bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 101,
  "eos_token_id": 102,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert-generation",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.42.4",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [33]:
torch.allclose(enc_embs[0], enc_embs.last_hidden_state)

True